In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/reduced/movies_m10_rich_pre.csv', index_col='movieId').iloc[:,2:]
df.head(1)

,cast,director,keywords,overview,title,genres,year
movieId,,,,,,,
117529,"['Chris Pratt', 'Bryce Dallas Howard', 'Irrfan...",Colin Trevorrow,"['monster', 'dna', 'tyrannosaurus rex', 'veloc...",Twenty-two years after the events of Jurassic ...,Jurassic World,"['Action', 'Adventure', 'Drama', 'Sci-Fi', 'Th...",2015


In [3]:
columns = ['cast', 'director', 'keywords', 'overview', 'title', 'genres', 'year']

In [4]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i for i in x)]
    else:
        if isinstance(x, str):
            return str.lower(x)
        else:
            return ''

In [5]:
def get_soup(x):
    return (''.join(x['cast']) + ' ' + x['director'] + ' ' + ''.join(x['keywords']) + ' ' + x['overview'] +
               ' ' + x['title'] + ' ' + ''.join(x['genres']))

In [6]:
for column in columns:
    df[column] = df[column].apply(clean_data)

In [7]:
df['soup'] = df.apply(get_soup, axis=1)
df.reset_index(inplace=True)

In [8]:
count = CountVectorizer(stop_words='english')

count_matrix = count.fit_transform(df['soup'])

cosim = cosine_similarity(count_matrix, count_matrix)

In [9]:
def get_recommendations(df, title, cosim):
    index = df[df['title'] == title+' '].index.values[0]

    sim_scores = list(enumerate(cosim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indexes = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indexes]



In [13]:
title = str.lower('Toy Story 2')

In [14]:
get_recommendations(df, title, cosim)

1146                                           toy story 
117                                          toy story 3 
1576                                      small soldiers 
168                                 being john malkovich 
1283                                                toys 
1952                                         stand by me 
1002    everything you always wanted to know about sex...
239                                       monsters, inc. 
1548                                                antz 
1691                                                 big 
Name: title, dtype: object

In [15]:
df['soup'].to_csv('soup.csv')